In [1]:
%run ..\notebooks\Util_func.ipynb

In [2]:
# # Bay Area
# REG = 'BayArea'
# base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\Processing_20200228\2_tour_extract\wt_wkday'
# raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\Processing_20200228\spatial_join'

# out_file = r'out\%s_7_Persons_AvgTNCTrips.xlsx' %REG

In [3]:
# # SANDAG
# REG = 'SANDAG'
# base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG\2_tour_extract\wt_wkday'
# raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG'

# out_file = r'out2\%s_7_Persons_AvgTNCTrips.xlsx' %REG

In [4]:
# SCAG
REG = 'SCAG'
base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SCAG_dataset_2020-02-27\2_tour_extract\wt_wkday'
raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SCAG_dataset_2020-02-27'

out_file = r'out2\%s_7_Persons_AvgTNCTrips.xlsx' %REG

In [5]:
## Process person records
per_df = pd.read_csv(join(base_dir,'survey2018_precx.dat'), sep=' ')
per_df = per_df[(per_df['psexpfac']>0)]
per_df['count'] = 1

## Process trip records
tmp_df = pd.read_csv(join(raw_dir, 'ex_trip_wZones.csv'))
tmp_df = tmp_df[['hh_id','person_num','trip_num','mode_type_imputed','mode_uber','mode_lyft']]
tmp_df = tmp_df.rename(columns={'hh_id':'hhno','person_num':'pno','trip_num':'tsvid'})

trip_df = pd.read_csv(join(base_dir,'survey2018_tripx.dat'), sep=' ')
trip_df = link_dt(trip_df)
trip_df = trip_df.loc[trip_df['mode']==9, ] # TNC trips only
trip_df = trip_df.merge(tmp_df, how='left')
trip_df = trip_df[trip_df['mode_type_imputed']!=4] #remove taxi trips

trip_df['tnc_type'] = 3 # prem/other
trip_df.loc[(trip_df['mode_uber']==1) | (trip_df['mode_lyft']==1), 'tnc_type'] = 1 # pooled
trip_df.loc[(trip_df['mode_uber']==2) | (trip_df['mode_lyft']==2), 'tnc_type'] = 2 # regular

trip_df = trip_df[(trip_df['trexpfac']>0)]
trip_df = trip_df[(trip_df['otaz']>0) & (trip_df['dtaz']>0)]
trip_df['count'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [6]:
col_dict = {
    'tncmode': {
        'desc': 'TNC_Mode',
        'col': 'tnc_type',
        'vals': range(1,4),
        'labels': ['2_Pool','1_Reg','3_PremOth']
    }
}

In [7]:
fname = out_file
writer = pd.ExcelWriter(fname, engine='xlsxwriter')
workbook = writer.book
format1 = workbook.add_format({'num_format': '#,##0.0'})
format2 = workbook.add_format({'num_format': '0.000'})

In [8]:
from xlsxwriter.utility import xl_rowcol_to_cell

row = 0
sname = 'Weekday'
d1_dict = col_dict['tncmode']
PREF = 'Daily Avg TNC Trips'

tab = prep_data_1d(trip_df, d1_dict['desc'],d1_dict['col'], 'trexpfac', d1_dict['vals'], d1_dict['labels'])

def prep_avg(suff, num_per):
    df = tab[[d1_dict['desc']+'_'+suff]]
    df['persons_'+suff] = num_per
    df['AvgDaily_TNCTrips'] = df[d1_dict['desc']+'_'+suff]/df['persons_'+suff]
    return df

title = PREF + ' (Unweighted)'
out_tab = prep_avg('samp', len(per_df))
row = write_to_excel(out_tab, sname, title, row)

title = PREF + ' (Weighted)'
out_tab = prep_avg('wtd', sum(per_df['psexpfac']))
row = write_to_excel(out_tab, sname, title, row)

tab_range = xl_rowcol_to_cell(0,1) + ':' + xl_rowcol_to_cell(0,2)
_ = writer.sheets[sname].set_column(tab_range, 11, format1)
tab_range = xl_rowcol_to_cell(0,3) + ':' + xl_rowcol_to_cell(0,3)
_ = writer.sheets[sname].set_column(tab_range, 11, format2)

In [9]:
writer.save()